In [54]:
openai_api_key = ""
gigachat_api_key = ""
data_path = 'data/RuBQ_2.0_paragraphs.json'

# Устанавливаем необходимые зависимости

In [2]:
! pip -q install langchain
! pip -q install datasets
! pip -q install faiss-cpu
! pip -q install sentence-transformers

In [3]:
! pip -q install openai

In [4]:
! pip -q install tiktoken

# Импортируем библиотеки

In [55]:
import time
import uuid
import requests
import os
from langchain.llms.base import LLM
from typing import Any, List, Mapping, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from langchain.document_loaders import DataFrameLoader
from langchain.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings
import pandas as pd
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.chains import LLMChain, StuffDocumentsChain
from langchain.prompts.example_selector import LengthBasedExampleSelector

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

# Пишем кастомный класс получения токена авторизации для GigaChat

In [56]:
class GigaChatSecureToken:
    access_token: str
    expires_at: int
    _offset: int = 60  # If token will be expired in {offset} seconds, is_expired() return true

    def __init__(self, access_token: str, expires_at: int):
        self.access_token = access_token
        self.expires_at = expires_at

    def is_expired(self):
        return round(time.time() * 1000) > self.expires_at + self._offset * 1000

# Определяем кастомный класс для работы с API GigaChat

In [57]:
class GigaChatLLM(LLM):
    api_key: str = None
    temperature: float = 0.7
    secure_token: GigaChatSecureToken = None

    @property
    def _llm_type(self) -> str:
        return "gigachat"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        if not self.secure_token or self.secure_token.is_expired():
            print("Obtaining new secure token")
            self._auth()
            if not self.secure_token or self.secure_token.is_expired():
                # New token was not obtained
                print("ERROR: new token was not updated, cannot call LLM")
                return ""
        headers = {
            "Authorization": f"Bearer {self.secure_token.access_token}",
            "Content-Type": "application/json"
        }
        req = {
            "model": "GigaChat:latest",
            "messages": [{
                "role": "user",
                "content": prompt
            }],
            "temperature": self.temperature
        }
        response = requests.post("https://gigachat.devices.sberbank.ru/api/v1/chat/completions", headers=headers, json=req, verify=False)
        if response.status_code != 200:
            print(f"ERROR: LLM call failed, status code: {response.status_code}")
            return ""
        return response.json()["choices"][0]["message"]["content"]

    def _auth(self):
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "RqUID": str(uuid.uuid4()),
            "Content-Type": "application/x-www-form-urlencoded"
        }

        scope_info = {"scope": "GIGACHAT_API_PERS"}
        response = requests.post("https://ngw.devices.sberbank.ru:9443/api/v2/oauth", data=scope_info, headers=headers, verify=False)
        if response.status_code != 200:
            print(f"ERROR: Something went wrong while obtaining secure token, status code: {response.status_code}")
            return
        content = response.json()

        expires_at = content["expires_at"]
        token = content["access_token"]
        if not (expires_at and token):
            print("ERROR: server returns empty values for fields 'expires_at' or 'access_token'")
            return
        self.secure_token = GigaChatSecureToken(token, expires_at)


llm = GigaChatLLM(api_key=gigachat_api_key)

In [58]:
llm(prompt="Пусть a = 5.  Сколько будет a в квадрате")

Obtaining new secure token


'Если мы возьмём число 5 и возведём его в квадрат, то получим 25.'

# Загружаем датасет

In [59]:
dataframe = pd.read_json(data_path)

In [60]:
dataframe

,uid,ru_wiki_pageid,text
0,0,58311,ЦСКА — советский и российский профессиональный...
1,1,58311,В первом сезоне в составе Континентальной хокк...
2,2,58311,В межсезонье 1992 года «армейскую» команду пок...
3,3,58311,"Однако ни Тихонов, ни Гущин, не согласились с ..."
4,4,58311,ЦСКА Александра Волчкова сезон 1996/97 провел ...
...,...,...,...
56947,56947,2562355,Кузнец Вакула (соч. 14) ― опера Петра Чайковск...
56948,56948,76411,"Трое приятелей: Джи, Харрис и Джордж, устав от..."
56949,56949,271647,Корри́да (исп. Corrida) — наиболее распростран...
56950,56950,160718,"События, которые легли в основу сценария, до и..."


In [61]:
dataframe = dataframe.dropna()
dataframe = dataframe[['text']]

In [62]:
dataframe

,text
0,ЦСКА — советский и российский профессиональный...
1,В первом сезоне в составе Континентальной хокк...
2,В межсезонье 1992 года «армейскую» команду пок...
3,"Однако ни Тихонов, ни Гущин, не согласились с ..."
4,ЦСКА Александра Волчкова сезон 1996/97 провел ...
...,...
56947,Кузнец Вакула (соч. 14) ― опера Петра Чайковск...
56948,"Трое приятелей: Джи, Харрис и Джордж, устав от..."
56949,Корри́да (исп. Corrida) — наиболее распростран...
56950,"События, которые легли в основу сценария, до и..."


# Трансформируем датафрейм в датафреймлоадер langchain. page_content_column в нашем случае поле text, так как именно над ним мы будем проводить векторизацию

In [63]:
loader = DataFrameLoader(dataframe, page_content_column = 'text')
data = loader.load()

In [64]:
data[:5]

[Document(page_content='ЦСКА — советский и российский профессиональный хоккейный клуб из Москвы, выступающий в Континентальной хоккейной лиге. Основан в 1946 году под названием ЦДКА (Центральный дом Красной Армии). В 1951 году переименован в ЦДСА (Центральный дом Советской Армии), а в 1954 в ЦСК МО (Центральный спортивный клуб Министерства обороны), под которым выступал до 1959 года, и с тех пор носит название ЦСКА (Центральный Спортивный Клуб Армии).'),
 Document(page_content='В первом сезоне в составе Континентальной хоккейной лиги ЦСКА выиграл дивизион Тарасова, но в плей-офф с трудом обыграл «Ладу» (3-2 по сумме встреч) и всухую проиграл «Динамо» (0-3). В конце сезона тренерский тандем Быков-Захаркин покинул команду, аргументировав своё решение желанием сосредоточиться на работе в сборной России, однако уже через несколько недель подписали контракт с командой «Салават Юлаев», таким образом продолжив совмещать работу в сборной и в клубе.'),
 Document(page_content='В межсезонье 1992 

In [65]:
token_counts_max = dataframe['text'].str.split().str.len().max()

# Определяем модель для эмбеддингов

In [67]:
# embedding_type = 'openai'
embedding_type = 'cointegrated/LaBSE-en-ru'

if embedding_type == 'openai':
    openai_batch_size = int(150000 / token_counts_max)
    embeddings = OpenAIEmbeddings(openai_api_key = openai_api_key, chunk_size = openai_batch_size)
else:
    embeddings = HuggingFaceEmbeddings(model_name=embedding_type)

# Загружаем эмбеддинги в векторную бд

In [69]:
from langchain import FAISS
from faiss import IndexHNSWFlat
from time import sleep
from tqdm import tqdm

index_name = f'{data_path}_{embedding_type}'

index = IndexHNSWFlat()
faiss = FAISS(embeddings, index, None, None)

try:
    db = FAISS.load_local(index_name, embeddings)
except:
    db = None

if db is None:
    if embedding_type == 'openai':
        for batch_number in tqdm(range(0, len(data), openai_batch_size)):
            batch = data[batch_number: batch_number + openai_batch_size]
            batch_db = faiss.from_documents(batch, embeddings)
            if db is None:
                db = batch_db
            else:
                db.merge_from(batch_db)

            sleep(20)
    else:
        db = faiss.from_documents(data, embeddings)

    db.save_local(index_name)

In [70]:
query = "Кто изобрел телефон?"
db.similarity_search(query, k=5)

[Document(page_content='В 1861 году немецкий физик и изобретатель Иоганн Филипп Рейс продемонстрировал другое устройство, которое также могло передавать музыкальные тона и человеческую речь по проводам. Аппарат имел микрофон оригинальной конструкции, источник питания (гальваническую батарею, или - "местную батарею" МБ) и динамик. Сам Рейс назвал сконструированное им устройство Telephone.'),
 Document(page_content='В 1876 году он получил патент США № 174465, описывающий «метод и аппарат… для передачи речи и других звуков по телеграфу… с помощью электрических волн». Фактически речь шла о телефоне. Кроме того, Белл вёл работы по использованию в телекоммуникации светового луча — направление, впоследствии приведшее к созданию волоконно-оптических технологий.'),
 Document(page_content='Александр Белл (1847—1922), американский учёный, изобретатель и бизнесмен шотландского происхождения, один из основоположников телефонии.'),
 Document(page_content='Первый проект телефона, носивший кодовое наз

# Определяем ретривер - верхнеуровневую обертку над similarity_search, в которую захардкожены какие-то параметры (в нашем случае 5 ближайших соседей)

In [71]:
retriever = db.as_retriever(search_kwargs = {"k": 5})
retriever.get_relevant_documents(query)

[Document(page_content='В 1861 году немецкий физик и изобретатель Иоганн Филипп Рейс продемонстрировал другое устройство, которое также могло передавать музыкальные тона и человеческую речь по проводам. Аппарат имел микрофон оригинальной конструкции, источник питания (гальваническую батарею, или - "местную батарею" МБ) и динамик. Сам Рейс назвал сконструированное им устройство Telephone.'),
 Document(page_content='В 1876 году он получил патент США № 174465, описывающий «метод и аппарат… для передачи речи и других звуков по телеграфу… с помощью электрических волн». Фактически речь шла о телефоне. Кроме того, Белл вёл работы по использованию в телекоммуникации светового луча — направление, впоследствии приведшее к созданию волоконно-оптических технологий.'),
 Document(page_content='Александр Белл (1847—1922), американский учёный, изобретатель и бизнесмен шотландского происхождения, один из основоположников телефонии.'),
 Document(page_content='Первый проект телефона, носивший кодовое наз

# Определяем шаблонизатор промпта

In [72]:
prefix = """
Ответь на вопрос: {query}

У меня есть некоторая информация, которая может тебе помочь. Если она релевантна вопросу - используй ее в качестве источника правды:
"""

context_example_template = """
{context}
"""

suffix = ""

# Определяем пайплайн промпт template -> языковая модель

# Объединяем все в одну функцию

In [73]:
from typing import Tuple


def get_answer(question: str) -> Tuple[str, str]:
  res = retriever.get_relevant_documents(question)
  examples = [{"context": doc.page_content} for doc in res ]
  context_example_template_prompt = PromptTemplate(
      input_variables=["context"],
      template=context_example_template
  )

  example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=context_example_template_prompt,
    max_length=900
  )

  few_shot_prompt_template_prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt=context_example_template_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
  )

  chain = few_shot_prompt_template_prompt | llm

  answer = chain.invoke({ "query": question })


  prompt = few_shot_prompt_template_prompt.format(query = question)
  return answer, prompt

In [74]:
answer, prompt = get_answer("Кто был подлинным изобретателем телефона?")
print(prompt)
print(answer)


Ответь на вопрос: Кто был подлинным изобретателем телефона?

У меня есть некоторая информация, которая может тебе помочь. Если она релевантна вопросу - используй ее в качестве источника правды:


1889 — Антонио Меуччи (р. 1808), итальянский учёный, признанный подлинным изобретателем телефона.


1808 — Антонио Меуччи (ум. 1889), итальянский учёный, подлинный изобретатель телефона.


В 1861 году немецкий физик и изобретатель Иоганн Филипп Рейс продемонстрировал другое устройство, которое также могло передавать музыкальные тона и человеческую речь по проводам. Аппарат имел микрофон оригинальной конструкции, источник питания (гальваническую батарею, или - "местную батарею" МБ) и динамик. Сам Рейс назвал сконструированное им устройство Telephone.


Первый проект телефона, носивший кодовое название «Purple One», не был доведён до конца.


Александр Белл (1847—1922), американский учёный, изобретатель и бизнесмен шотландского происхождения, один из основоположников телефонии.

Подлинным изобр

# Сравним аугментированную модель с неаугментированной

In [75]:
questions = [
    "Кто был подлинным создателем первого телефона?",
    "Как называется столица США?",
    "Кто написал Войну и мир?",
    "Почему трава зеленая?",
    "Что такое водяной вор, клипсидра?"
]

In [76]:
compare_df = pd.DataFrame({"gigachat": [], "gigachat + augmentation": [], "prompt": []})
for question in tqdm(questions):
  llm_answer = llm(question)
  llm_augmented_answer, llm_augmented_prompt = get_answer(question)

  concat_df = pd.DataFrame.from_dict({"gigachat": [llm_answer], "gigachat + augmentation": [llm_augmented_answer], "prompt": [llm_augmented_prompt]})

  compare_df = pd.concat([compare_df, concat_df], ignore_index = True)
  if embedding_type == 'openai':
      sleep(20)

100%|██████████| 5/5 [00:16<00:00,  3.33s/it]


In [ ]:
pd.set_option('display.max_colwidth', None)

In [77]:
compare_df

,gigachat,gigachat + augmentation,prompt
0,Подлинным создателем первого телефона считаетс...,Подлинным создателем первого телефона считаетс...,\nОтветь на вопрос: Кто был подлинным создател...
1,Столицей Соединённых Штатов Америки является г...,Столицей США является город Вашингтон.,\nОтветь на вопрос: Как называется столица США...
2,"""Война и мир"" был написан Львом Николаевичем Т...",Война и мир был написан Львом Николаевичем Тол...,\nОтветь на вопрос: Кто написал Войну и мир?\n...
3,Зеленый цвет растениям придаёт хлорофилл — зел...,"Трава зеленая из-за наличия в ней хлорофилла, ...",\nОтветь на вопрос: Почему трава зеленая?\n\nУ...
4,Водяной вор (или клипсидра) — это устройство д...,"Извините, но я не могу найти информацию о ""вод...","\nОтветь на вопрос: Что такое водяной вор, кли..."


In [78]:
compare_df.to_html('answer-local-embedding.html')